In [36]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#MongoCRUSD
from mongo import MongoCRUSD

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = MongoCRUSD(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Michael Reynolds - Project Two')

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.A( 
        href="https://www.snhu.edu",
        children=[
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'width': '150px', 'height': '150px'}
            )
        ]
    )),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.H3('by Michael Reynolds'))),
    html.Hr(),
    html.Div(
        children=[
            html.H1("Filter"),
            dcc.Dropdown(['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset'], value='Reset', id='filter-type',
            multi=False,
            clearable=True,
            style={"width": "50%"}),
        ]
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        column_selectable='single',
        row_selectable='single',
        selected_rows=[0],
        selected_columns=[],
        page_size=10

    ),
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id','data'),
    [Input('filter-type', 'value')]
)

def update_dashboard(filter_type):
    breeds = []
    
    if filter_type == "Water Rescue":
        
        breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
        df=pd.DataFrame.from_records(db.read({"breed": {"$in": breeds}}))
        
    elif filter_type == "Mountain Rescue":
        
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
        df=pd.DataFrame.from_records(db.read({"breed": {"$in": breeds}}))
        
    elif filter_type == "Disaster Rescue":
        
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        df=pd.DataFrame.from_records(db.read({"breed": {"$in": breeds}}))
        
    else:
        df=pd.DataFrame.from_records(db.read({}))
    
    df.drop(columns=['_id'],inplace=True)
    
    data=df.to_dict('records')
      
      
    return data
            
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')]
)

def update_graphs(viewData, filterData):
    if viewData is None:
        return []
    
    if filterData == 'Reset':
    
        df = pd.DataFrame(viewData)
        breeds = df['breed'].value_counts()
        breed_count = breeds.sort_values(ascending=False)


        common_breeds = breed_count.head(10)


        other_breeds = breed_count[10:].sum()
        common_breeds['Other'] = other_breeds

        piechart = px.pie(values=common_breeds.values, names=common_breeds.index)

        return [dcc.Graph(figure=piechart)]
    
    else:
        
        piechart = px.pie(viewData, names='breed')
        
        return [dcc.Graph(figure=piechart)]
    
#This callback will highlight a cell on the data table when the user selects it

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns'),
     Input('datatable-id', 'selected_rows')]
)

def update_styles(selected_columns, selected_rows):
    styles = []
    
    if selected_columns and selected_rows:
        
        row = selected_rows[0]  # Assuming single row selection
        column = selected_columns[0]  # Assuming single column selection
        styles.append({'if': {'row_index': row, 'column_id': column}, 'background_color': '#D2F3FF'})
        
        return styles
    
    else:
        styles += [
        {
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns] 

        styles += [
        {
            'if': { 'row_index': row},
            'background_color': '#D2F3FF'
        } for row in selected_rows]

        return styles


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9]),
                    html.H1("Outcome Type"),
                    html.P(dff.iloc[row,11])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:16951/
